## Задание 1
Есть ряд кампаний с определенным настройками (таргет, частота показа
креатива). Рекламодатель создает кампанию с целью получить конверсию, но
оплачивает клики. Существует гипотеза, что CTR по кампании снижается
пропорционально возрасту кампании. Каким образом и какими методами вы бы
подтверждали/опровергали данную гипотезу? На какие нюансы обратили бы
внимание?

* Можем просто проверить динамику изменения CTR по каждой из кампаний за период
* Динамику изменения среднего CTR по всем кампаниям за период

Если рост всех очевиден - сворачиваем лавочку, всё понятно, если нет - делаем по-красивому: выдвигаем нулевую, альтернативные гипотезы и занимаемся.

Но для того, чтобы выдвинуть гипотезу вначале нужно вообще понять - а используя какие данные нам вообще можно сделать вывод/выдвинуть гипотезу, что CTR снижается/наоборот.

Вообще, хорошее понимание растёт что-то/убывает даёт коэффициент угла наклона прямой. Потому при помощи регрессии и имеющихся данных получаем уравнение прямой для каждой кампании вида:

$$ y = kx + b, \text{где}$$

$$ k \text{ - интересующий нас показатель возрастания/убывания} $$

Знаем, что если k < 0 - CTR убывает, k > 0 - возрастает.

По идее, распределение значений k, которое построим - будет нормальным. Ну и уже далее выдвигаем нулевую и альтернативные гипотезы, принимаем $\alpha$ за 0.05, расчитываем уровень значимости и, соответственно отвергаем или принимаем нулевую гипотезу - делаем выводы о динамике CTR.

## Задание 2
Есть 4 формата трафика (a, b, c, d). Существуют определенные индикаторы X и
Y, которые описывают источник (source) трафика. Также, каждому формату
соответствует определенный threshold, при пересечении которого источник, с
которого пришел трафик, считается фродовым, то есть (в общем виде)

Необходимо написать код, который будет присваивать значение фрода
источнику в зависимости от его формата и индикаторов и сохранять в
отдельный столбец датафрейма

![title](task_cond.png)

In [1]:
import pandas as pd
import numpy as np
import random

В первую очередь переведу листы пороговых значений для каждого формата в словарь для удобства, потому что формат хранения данных в виде list(a = tx_a, b = tx_b, etc) невозможен, а индексация по списку будет неудобна

In [2]:
#словари со случайно выбранными пороговыми значениями
format_thresh_x = {
    'a':37,
    'b':50,
    'c':3
}
format_thresh_y = {
    'a':22,
    'b':78,
    'c':18
}

Создам схожий датафрейм и выполню реализацию на нём

In [3]:
#датафрейм со случайными данными
np.random.seed(1)
size = 10
df = pd.DataFrame({'source': [i for i in range(10)],
                'X': np.random.randint(low=0, high=100, size=size),
                'Y': np.random.randint(low=0, high=100, size=size) ,
                'format': [np.random.choice(list(format_thresh_x.keys())) for i in range(size)]})

In [4]:
df

,source,X,Y,format
0,0,37,76,b
1,1,12,71,c
2,2,72,6,c
3,3,9,25,a
4,4,75,50,b
5,5,5,20,b
6,6,79,18,c
7,7,64,84,a
8,8,16,11,c
9,9,1,28,b


Правило возьму то же

In [5]:
df["fraud_value"] = np.where(
    #первое условие
   (df.X > np.array([format_thresh_x[key] for key in df.format])) & 
        (df.Y > np.array([format_thresh_y[key] for key in df.format])), 
   "fraud", 
    #второе условие
    np.where((df.X > np.array([format_thresh_x[key] for key in df.format])) | 
        (df.Y > np.array([format_thresh_y[key] for key in df.format])),
        "suspicious", 
        "not fraud")
)

Что имеем в итоге:

In [6]:
df

,source,X,Y,format,fraud_value
0,0,37,76,b,not fraud
1,1,12,71,c,fraud
2,2,72,6,c,suspicious
3,3,9,25,a,suspicious
4,4,75,50,b,suspicious
5,5,5,20,b,not fraud
6,6,79,18,c,suspicious
7,7,64,84,a,fraud
8,8,16,11,c,suspicious
9,9,1,28,b,not fraud


## Задание 3
Предположим есть база данных со следующими полями
date, publisher (char), advertiser (char), ip (char), event (enum)

Event - это событие, которое происходит. Например,
* 0 - ошибка при открытии рекламы,
* 1 - показ
* 2 - клик
* 3 - конверсия

Необходимо составить псевдо SQL-запросы следующего типа:
* А. Вывести список из 10 паблишеров, у которых наилучшее соотношение конверсия / клик за последнюю неделю
* Б. Вывести уникальные сочетания паблишеров и адвертайзеров, у которых хотя бы единожды за день соотношение ошибок и показов было больше, чем среднее значение соотношения ошибок и показов за все время

In [7]:
a_query = '''
SELECT publisher 
FROM (
    SELECT publisher, 
        countIf(event=3) as num_conv, 
        countIf(event=2) as num_clicks, 
        num_conv/num_clicks as cr
    FROM db
    WHERE date BETWEEN (today() AND date_sub(DAY, 7, today()))
    GROUPBY publisher
    ORDER BY cr DESC
    LIMIT 10
)
'''

In [8]:
b_query = '''
SELECT DISTINCT publisher, advertiser
FROM (
    SELECT publisher, advertiser, date,
        countIf(event=0) as num_errors,
        countIf(event=1) as num_imps,
        num_errors/num_imps as error_imps_rate,
        (SELECT countIf(event=0)/countIf(event=1) FROM db) as avg_error_imps_rate
    FROM db
    GROUPBY publisher, advertiser, date
    HAVING error_imps_rate > avg_error_imps_rate
)
'''